## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)



In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPyDatabase.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Meulaboh,ID,4.1363,96.1285,77.49,87,86,3.20,light rain
1,1,Yellowknife,CA,62.4560,-114.3525,28.42,67,75,21.85,broken clouds
2,2,Bluff,NZ,-46.6000,168.3333,56.66,76,51,21.32,broken clouds
3,3,Ushuaia,AR,-54.8000,-68.3000,53.26,58,20,0.00,few clouds
4,4,Port Hardy,CA,50.6996,-127.4199,39.38,81,100,10.36,light intensity shower rain


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 75


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
pref_temp_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
pref_temp_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
5,5,Malanje,AO,-9.5402,16.3410,65.35,96,100,2.04,moderate rain
28,28,Saldanha,ZA,-33.0117,17.9442,68.05,50,0,10.76,clear sky
33,33,Isiolo,KE,0.3546,37.5822,71.01,64,11,5.97,few clouds
41,41,Castro,BR,-24.7911,-50.0119,70.83,74,88,2.93,overcast clouds
51,51,Chuy,UY,-33.6971,-53.4616,70.11,74,34,10.56,scattered clouds
55,55,Jiexiu,CN,37.0244,111.9125,65.59,22,0,6.42,clear sky
68,68,Dehloran,IR,32.6941,47.2679,71.26,29,0,9.06,clear sky
77,77,Wanning,CN,18.8003,110.3967,67.50,89,72,4.41,broken clouds
80,80,Dingle,PH,10.9995,122.6711,74.95,90,100,1.14,moderate rain
85,85,Venice,US,27.0998,-82.4543,73.38,54,100,18.41,haze


In [7]:
# 4a. Determine if there are any empty rows.
pref_temp_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [ ]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

#None empty

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = pref_temp_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Malanje,AO,65.35,moderate rain,-9.5402,16.3410,
28,Saldanha,ZA,68.05,clear sky,-33.0117,17.9442,
33,Isiolo,KE,71.01,few clouds,0.3546,37.5822,
41,Castro,BR,70.83,overcast clouds,-24.7911,-50.0119,
51,Chuy,UY,70.11,scattered clouds,-33.6971,-53.4616,
55,Jiexiu,CN,65.59,clear sky,37.0244,111.9125,
68,Dehloran,IR,71.26,clear sky,32.6941,47.2679,
77,Wanning,CN,67.50,broken clouds,18.8003,110.3967,
80,Dingle,PH,74.95,moderate rain,10.9995,122.6711,
85,Venice,US,73.38,haze,27.0998,-82.4543,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [12]:
# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("No hotel, skip")
        

No hotel, skip
No hotel, skip
No hotel, skip
No hotel, skip
No hotel, skip
No hotel, skip
No hotel, skip
No hotel, skip


In [33]:
# 7. Drop the rows where there is no Hotel Name.
nan_value = float("NaN")
hotel_df = hotel_df.replace("", nan_value)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()
                   

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Malanje,AO,65.35,moderate rain,-9.5402,16.3410,Hotel Ginga
28,Saldanha,ZA,68.05,clear sky,-33.0117,17.9442,Blue Bay Lodge
33,Isiolo,KE,71.01,few clouds,0.3546,37.5822,Gaddisa Hotel
41,Castro,BR,70.83,overcast clouds,-24.7911,-50.0119,CHACARA BAILLY
51,Chuy,UY,70.11,scattered clouds,-33.6971,-53.4616,Nuevo Hotel Plaza


In [32]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [34]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [35]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.8)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))